# Agentic AI with PydanticAI: Sales Q&A with Self-Reflection

This notebook demonstrates a comprehensive two-agent flow for answering and self-correcting sales-related questions with detailed step-by-step logging:

####################### ADVANTAGES OF HAVING REFLECTION AGENTS OR AGENT AS A JUDGE ###########################
How simply relying on LLM
Explain the Agent Ouputs with calm

## 🎯 **Agent Architecture Overview**

<img src="reflection-agent.png" width="50%">


### 1. **Primary Sales Agent**  
- **Role**: Provides initial comprehensive sales answers
- **Capability**: Handles complex sales scenarios, strategies, and best practices
- **Output**: Structured responses with detailed explanations

### 2. **Reflection & Quality Agent**  
- **Role**: Reviews, validates, and improves the initial answer
- **Capability**: Identifies gaps, inaccuracies, and enhancement opportunities
- **Output**: Refined and optimized sales advice

## 🔧 **Setup Requirements**

Make sure you have:
1. Set `OPENAI_API_KEY` in your `.env` file (provided in this project)
2. Installed all required dependencies (see installation cell below)
3. Python 3.10+ environment

## 📊 **What You'll Learn**

- How to create specialized sales AI agents using PydanticAI
- Implement agent reflection patterns for quality assurance
- Structure complex sales scenarios with proper validation
- Monitor and log agent decision-making processes
- Handle real-world sales challenges with AI assistance

## 🚀 **Step 1: Install Dependencies**

Install all required packages for our sales agents system. This includes:

- **pydantic-ai**: The core framework for building AI agents with structured outputs
- **python-dotenv**: For loading environment variables from .env file
- **openai**: OpenAI API client for GPT model access
- **rich**: For beautiful console output and logging
- **datetime**: For timestamping our agent operations

```bash
!pip install pydantic-ai python-dotenv openai rich
```

### 📋 **Package Explanations**

- **pydantic-ai**: Provides type-safe AI agents with validation
- **python-dotenv**: Securely manages API keys and configuration
- **openai**: Direct interface to OpenAI's GPT models
- **rich**: Enhanced console output for better debugging and logging

## 🔧 **Step 2: Comprehensive Imports & Environment Setup**

This section sets up our complete development environment:

### 🔍 **What we're importing:**
- **Core Libraries**: For basic functionality and environment management
- **PydanticAI Components**: For agent creation and model integration  
- **Logging & Display**: For detailed step-by-step output
- **Type Annotations**: For better code documentation and validation

### 🎯 **Environment Configuration:**
- Load API keys from `.env` file
- Configure logging levels for detailed output
- Set up console formatting for better readability

In [17]:
# ========================================
# CORE IMPORTS & ENVIRONMENT SETUP
# ========================================

print("🔄 Loading environment and dependencies...")

# Core Python libraries
import os
from datetime import datetime
from typing import Optional, List

# Environment management
from dotenv import load_dotenv
print("✅ Loading environment variables from .env file...")
load_dotenv()

# Validate critical environment variables
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found in environment!")
    print("📝 Please set your OpenAI API key in the .env file")
else:
    print("✅ OpenAI API key loaded successfully")

# PydanticAI imports
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIModel

# Apply nest_asyncio to allow nested event loops
import nest_asyncio
nest_asyncio.apply() 

# Rich for beautiful console output
try:
    from rich.console import Console
    from rich.panel import Panel
    from rich.table import Table
    from rich import print as rprint
    console = Console()
    print("✅ Rich console formatting enabled")
except ImportError:
    print("⚠️  Rich not available, using standard print")
    console = None
    rprint = print

print("🎯 All imports completed successfully!")
print(f"🕐 Setup completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

🔄 Loading environment and dependencies...
✅ Loading environment variables from .env file...
✅ OpenAI API key loaded successfully
✅ Rich console formatting enabled
🎯 All imports completed successfully!
🕐 Setup completed at: 2025-06-13 10:02:50


## 📋 **Step 3: Define Structured Output Schemas**

Here we define the data structures that our sales agents will use for their responses. Using Pydantic models ensures:

### 🔍 **Schema Benefits:**
- **Type Safety**: Ensures all outputs follow the expected format
- **Validation**: Automatic validation of agent responses
- **Documentation**: Self-documenting code with field descriptions
- **Consistency**: Standardized output across all agent interactions

### 📊 **Schema Structure:**
Our schemas capture different aspects of sales responses:
- **Basic sales answer**: Core response with confidence scoring
- **Detailed sales analysis**: Comprehensive breakdown with metrics
- **Reflection metadata**: Quality assessment and improvement suggestions

In [18]:
# ========================================
# DEFINE STRUCTURED OUTPUT SCHEMAS
# ========================================

print("🔧 Creating structured output schemas for sales agents...")

class SalesAnswer(BaseModel):
    """
    Primary response schema for sales-related questions.
    Includes comprehensive answer with metadata for tracking.
    """
    answer: str = Field(
        description="Comprehensive sales answer or advice",
        min_length=50
    )
    confidence_score: float = Field(
        description="Confidence level (0.0 to 1.0) in the provided answer",
        ge=0.0,
        le=1.0,
        default=0.8
    )
    key_points: List[str] = Field(
        description="Main takeaways or action items from the answer",
        default_factory=list
    )
    sales_category: str = Field(
        description="Category of sales topic addressed",
        default="general"
    )

print("✅ SalesAnswer schema created with fields:")
print("   - answer: Main response content")
print("   - confidence_score: AI confidence level")
print("   - key_points: Extracted action items")
print("   - sales_category: Topic classification")

class ReflectionAnalysis(BaseModel):
    """
    Schema for reflection agent responses.
    Provides detailed analysis and improvement suggestions.
    """
    reviewed_answer: str = Field(
        description="Improved or validated version of the original answer"
    )
    quality_score: float = Field(
        description="Quality assessment of original answer (0.0 to 1.0)",
        ge=0.0,
        le=1.0
    )
    improvements_made: List[str] = Field(
        description="List of specific improvements or corrections made",
        default_factory=list
    )
    missing_elements: List[str] = Field(
        description="Important aspects that were missing from original answer",
        default_factory=list
    )
    additional_considerations: Optional[str] = Field(
        description="Extra insights or considerations for the sales scenario",
        default=None
    )

print("✅ ReflectionAnalysis schema created with fields:")
print("   - reviewed_answer: Refined response")
print("   - quality_score: Original answer assessment")
print("   - improvements_made: Specific enhancements")
print("   - missing_elements: Gaps identified")
print("   - additional_considerations: Extra insights")

print("🎯 Schema definitions completed!")
print("="*60)


🔧 Creating structured output schemas for sales agents...
✅ SalesAnswer schema created with fields:
   - answer: Main response content
   - confidence_score: AI confidence level
   - key_points: Extracted action items
   - sales_category: Topic classification
✅ ReflectionAnalysis schema created with fields:
   - reviewed_answer: Refined response
   - quality_score: Original answer assessment
   - improvements_made: Specific enhancements
   - missing_elements: Gaps identified
   - additional_considerations: Extra insights
🎯 Schema definitions completed!


## 🤖 **Step 4: Initialize the Primary Sales Agent**

Now we create our first agent - the Primary Sales Agent. This agent will:

### 🎯 **Agent Capabilities:**
- **Expert Knowledge**: Comprehensive understanding of sales strategies, techniques, and best practices
- **Structured Output**: Returns responses using our `SalesAnswer` schema
- **Context Awareness**: Adapts responses based on sales scenarios and contexts
- **Professional Guidance**: Provides actionable advice for real-world sales situations

### 🔧 **Configuration Details:**
- **Model**: GPT-3.5-turbo for balanced performance and cost
- **Output Schema**: `SalesAnswer` with confidence scoring and key points
- **System Role**: Professional sales expert and advisor
- **Temperature**: Controlled for consistent, professional responses

In [19]:
# ========================================
# INITIALIZE PRIMARY SALES AGENT  
# ========================================

print("🚀 Initializing Primary Sales Agent...")

# Configure the OpenAI model with environment variables
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-3.5-turbo")
model_temperature = float(os.getenv("OPENAI_MODEL_TEMPERATURE", "0.7"))

print(f"📋 Model Configuration:")
print(f"   - Model: {model_name}")
print(f"   - Temperature: {model_temperature}")
print(f"   - API Key: {'✅ Loaded' if os.getenv('OPENAI_API_KEY') else '❌ Missing'}")

model = OpenAIModel(
    model_name=model_name,
    provider=OpenAIProvider(api_key=os.getenv("OPENAI_API_KEY"))
)

# Create the Primary Sales Agent with comprehensive system prompt
sales_agent = Agent(
    model=model,
    output_type=SalesAnswer,
    system_prompt=(
        "You are an expert sales professional and business development specialist with over 15 years of experience. "
        "Your expertise spans B2B sales, enterprise sales, SaaS sales, consultative selling, and sales leadership. "
        "\n\nYour role is to provide comprehensive, actionable sales advice that includes:"
        "\n- Strategic sales approaches and methodologies"
        "\n- Practical techniques for prospecting, qualifying, and closing"
        "\n- Relationship building and customer success strategies"
        "\n- Objection handling and negotiation tactics"
        "\n- Sales process optimization and performance improvement"
        "\n\nAlways provide specific, implementable advice with clear action items. "
        "Include confidence levels and categorize your responses appropriately. "
        "Focus on ethical, relationship-based selling approaches that create mutual value."
    )
)

print("✅ Primary Sales Agent initialized successfully!")
print("🎯 Agent configured with:")
print("   - Output Type: SalesAnswer (structured response)")
print("   - System Role: Expert Sales Professional")
print("   - Focus: Comprehensive, actionable sales advice")
print("="*60)

🚀 Initializing Primary Sales Agent...
📋 Model Configuration:
   - Model: gpt-4o
   - Temperature: 0.7
   - API Key: ✅ Loaded
✅ Primary Sales Agent initialized successfully!
🎯 Agent configured with:
   - Output Type: SalesAnswer (structured response)
   - System Role: Expert Sales Professional
   - Focus: Comprehensive, actionable sales advice


## 💬 **Step 5: Test the Primary Sales Agent**

Let's test our Primary Sales Agent with a real-world sales scenario. We'll use a common sales challenge that many professionals face:

### 🎯 **Test Scenario:**
We'll ask about handling price objections - one of the most common challenges in sales.

### 🔍 **What to Observe:**
- **Structured Output**: Note how the response follows our `SalesAnswer` schema
- **Confidence Scoring**: See the AI's confidence in its advice
- **Key Points**: Actionable takeaways for immediate implementation
- **Categorization**: How the AI classifies the sales topic
- **Comprehensive Advice**: Detailed, practical guidance

### 📊 **Response Analysis:**
The agent will provide a comprehensive response that demonstrates its understanding of sales dynamics and ability to offer practical solutions.

In [20]:
# ========================================
# TEST PRIMARY SALES AGENT
# ========================================

print("💬 Testing Primary Sales Agent with real-world scenario...")
print("="*60)

# Define a comprehensive sales question
sales_question = """
How do I effectively handle price objections from enterprise clients who say 
our solution is too expensive compared to competitors? I need practical strategies 
that work in B2B SaaS sales where the deal sizes are typically $50K-200K annually.
"""

print(f"🎯 Sales Question:")
print(f"   {sales_question.strip()}")
print("\n🔄 Processing with Primary Sales Agent...")

# Run the sales agent and capture detailed timing
start_time = datetime.now()
primary_result = sales_agent.run_sync(sales_question) # PydanticAI agent.run_sync() is a synchronous wrapper for agent.run()
end_time = datetime.now()
processing_time = (end_time - start_time).total_seconds()

print(f"✅ Response generated in {processing_time:.2f} seconds")
print("="*60)

# Display structured output with rich formatting
if console:
    # Use Rich for beautiful output
    console.print("\n[bold green]🎯 PRIMARY SALES AGENT RESPONSE[/bold green]")
    
    # Create a table for the structured response
    table = Table(show_header=True, header_style="bold magenta")
    table.add_column("Field", style="cyan", width=20)
    table.add_column("Value", style="white")
    
    table.add_row("Confidence Score", f"{primary_result.output.confidence_score:.1%}")
    table.add_row("Sales Category", primary_result.output.sales_category)
    table.add_row("Key Points Count", str(len(primary_result.output.key_points)))
    
    console.print(table)
    
    # Display the main answer in a panel
    console.print(Panel(
        primary_result.output.answer,
        title="[bold blue]Comprehensive Sales Advice[/bold blue]",
        border_style="blue"
    ))
    
    # Display key points
    if primary_result.output.key_points:
        console.print("\n[bold yellow]🔑 KEY TAKEAWAYS:[/bold yellow]")
        for i, point in enumerate(primary_result.output.key_points, 1):
            console.print(f"   {i}. {point}")
else:
    # Fallback to standard print
    print("\n🎯 PRIMARY SALES AGENT RESPONSE:")
    print(f"Confidence Score: {primary_result.output.confidence_score:.1%}")
    print(f"Sales Category: {primary_result.output.sales_category}")
    print(f"\n📝 Answer:")
    print(primary_result.output.answer)
    
    if primary_result.output.key_points:
        print(f"\n🔑 Key Takeaways:")
        for i, point in enumerate(primary_result.output.key_points, 1):
            print(f"   {i}. {point}")

print("\n" + "="*60)
print("🔄 Ready for reflection analysis...")

💬 Testing Primary Sales Agent with real-world scenario...
🎯 Sales Question:
   How do I effectively handle price objections from enterprise clients who say 
our solution is too expensive compared to competitors? I need practical strategies 
that work in B2B SaaS sales where the deal sizes are typically $50K-200K annually.

🔄 Processing with Primary Sales Agent...
✅ Response generated in 13.43 seconds


🎯 PRIMARY SALES AGENT RESPONSE

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Value                                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Confidence Score     │ 90.0%                                                          │
│ Sales Category       │ Objection Handling, Negotiation Tactics, Relationship Building │
│ Key Points Count     │ 5                                                              │
└──────────────────────┴────────────────────────────────────────────────────────────────┘

╭────────────────────────────────────────── Comprehensive Sales Advice ───────────────────────────────────────────╮
│ Handling price objections effectively in enterprise B2B SaaS sales requires a strategic approach that focuses   │
│ on demonstrating value, understanding client needs, and negotiating terms that highlight the unique benefits of │
│ your solution. Here are several practical strategies you can use:                                               │
│                                                                                                                 │
│ 1. **Understand the Root Objection:** Begin by asking open-ended questions to uncover the underlying reasons    │
│ for their price concern. e.g., “Can you share more about what specifically seems too high compared to           │
│ competitors?” This helps you identify whether the objection is based on budget, perceived value, or comparison  │
│ with competitors.                                                                                               │
│                                                                                                                 │
│ 2. **Value Proposition Emphasis:** Clearly articulate and align your solution’s value with the client’s         │
│ specific business outcomes. Showcase case studies or data that demonstrate the ROI and cost savings achieved by │
│ businesses similar to theirs. This helps shift focus from price to the overall business impact.                 │
│                                                                                                                 │
│ 3. **Competitive Differentiation:** Highlight unique features, support, integration capabilities, scalability,  │
│ and any bespoke solutions your product offers that competitors do not. This reinforces why choosing your        │
│ solution is economically advantageous in the long run.                                                          │
│                                                                                                                 │
│ 4. **Bundle and Incentivize:** Offer bundled packages or limited-time incentives that add value without         │
│ significant cost reduction. For example, including premium support for the first year can sweeten the deal and  │
│ justify the pricing.                                                                                            │
│                                                                                                                 │
│ 5. **Risk Reduction:** Propose terms that minimize the client’s perceived risk, such as trial periods,          │
│ performance guarantees, or flexible contracts that allow for growth and adjustments over time.                  │
│                                                                                                                 │
│ 6. **Negotiate based on volume or long-term commitment:** Offer price adjustments for larger volume purchases   │
│ or longer contractual commitments, ensuring the client sees it as a win-win scenario.                           │
│                                                                                                                 │
│ 7. **Build Strong Relationships:** Develop rapport and trust with stakeholders through regular interactions,    │
│ allowing you to better understand their financial constraints and positively influence their purchasing         │
│ decisions over time.                                                                                            │
│                                                                                                                 │
│ By consistently demonstrating how your solution provides superior value that outweighs its cost, and creatively │
│ negotiating terms, you can effectively manage and overcome price objections.                                    │
│                                                       

🔑 KEY TAKEAWAYS:

...

2. Emphasize your value proposition and ROI.

3. Highlight unique features and differentiation.

4. Offer bundled packages or incentives.

5. Mitigate risk with trials or flexible terms.


🔄 Ready for reflection analysis...


## 🔍 **Step 6: Initialize the Reflection & Quality Agent**

Now we create our second agent - the Reflection & Quality Agent. This agent serves as a quality assurance layer that:

### 🎯 **Agent Purpose:**
- **Quality Review**: Evaluates the comprehensiveness and accuracy of the primary response
- **Gap Analysis**: Identifies missing elements or overlooked considerations
- **Enhancement**: Provides improved versions with additional insights
- **Validation**: Ensures advice is practical and implementable

### 🧠 **Advanced Capabilities:**
- **Critical Thinking**: Applies analytical review to sales advice
- **Industry Knowledge**: Cross-references best practices and methodologies
- **Completeness Check**: Ensures all aspects of the question are addressed
- **Improvement Suggestions**: Offers specific enhancements and optimizations

### 📊 **Output Schema:**
Uses `ReflectionAnalysis` schema for detailed quality assessment and improvement tracking.

In [21]:
# ========================================
# INITIALIZE REFLECTION & QUALITY AGENT
# ========================================

print("🔍 Initializing Reflection & Quality Agent...")

# Create the Reflection Agent with advanced quality assessment capabilities
reflection_agent = Agent(
    model=model,
    output_type=ReflectionAnalysis,
    system_prompt=(
        "You are a senior sales director and quality assurance specialist with 20+ years of experience "
        "in sales leadership, training, and methodology development. Your role is to critically review "
        "and enhance sales advice to ensure it meets the highest professional standards.\n\n"
        
        "Your responsibilities include:\n"
        "• QUALITY ASSESSMENT: Evaluate the accuracy, completeness, and practicality of sales advice\n"
        "• GAP ANALYSIS: Identify missing elements, overlooked strategies, or incomplete considerations\n"
        "• ENHANCEMENT: Improve responses with additional insights, methodologies, or best practices\n"
        "• VALIDATION: Ensure all advice is ethical, relationship-focused, and results-oriented\n"
        "• SCORING: Provide objective quality scores based on comprehensiveness and actionability\n\n"
        
        "Review each response through these lenses:\n"
        "1. Strategic completeness - Are all key aspects covered?\n"
        "2. Tactical specificity - Are the recommendations specific and actionable?\n"
        "3. Industry relevance - Does it address the specific context and industry?\n"
        "4. Ethical alignment - Does it promote value-based, relationship-centric selling?\n"
        "5. Practical implementation - Can the advice be immediately applied?\n\n"
        
        "Always provide constructive improvements and maintain a focus on long-term relationship building "
        "and mutual value creation in sales interactions."
    )
)

print("✅ Reflection & Quality Agent initialized successfully!")
print("🎯 Agent configured with:")
print("   - Output Type: ReflectionAnalysis (detailed assessment)")
print("   - System Role: Senior Sales Director & QA Specialist")
print("   - Focus: Quality assurance and continuous improvement")
print("="*60)

🔍 Initializing Reflection & Quality Agent...
✅ Reflection & Quality Agent initialized successfully!
🎯 Agent configured with:
   - Output Type: ReflectionAnalysis (detailed assessment)
   - System Role: Senior Sales Director & QA Specialist
   - Focus: Quality assurance and continuous improvement


## 🔄 **Step 7: Run Quality Analysis & Reflection**

Now we'll have our Reflection Agent analyze the primary response and provide comprehensive quality assessment:

### 🎯 **Reflection Process:**
1. **Quality Evaluation**: Assess the original response's comprehensiveness and accuracy
2. **Gap Identification**: Find missing elements or overlooked considerations
3. **Enhancement Generation**: Create improved version with additional insights
4. **Scoring**: Provide objective quality metrics
5. **Recommendations**: Offer specific improvement suggestions

### 📊 **Expected Outcomes:**
- **Quality Score**: Numerical assessment of the original response
- **Improved Answer**: Enhanced version with additional insights
- **Specific Improvements**: List of enhancements made
- **Missing Elements**: Identified gaps in the original response
- **Additional Considerations**: Extra insights for the sales scenario

### 🔍 **Analysis Focus:**
The reflection will evaluate strategic completeness, tactical specificity, industry relevance, and practical implementation.

In [22]:
# ========================================
# RUN QUALITY ANALYSIS & REFLECTION
# ========================================

print("🔄 Running Reflection & Quality Analysis...")
print("="*60)

# Prepare comprehensive reflection prompt
reflection_prompt = f"""
SALES QUESTION ANALYSIS REQUEST

Original Question:
{sales_question.strip()}

Primary Agent Response:
- Answer: {primary_result.output.answer}
- Confidence Score: {primary_result.output.confidence_score}
- Key Points: {primary_result.output.key_points}
- Sales Category: {primary_result.output.sales_category}

Please provide a comprehensive quality analysis and enhanced response following your role as a senior sales director and QA specialist.
"""

print("🔍 Analyzing primary response for quality and completeness...")

# Run reflection analysis with timing
reflection_start = datetime.now()
reflection_result = reflection_agent.run_sync(reflection_prompt)
reflection_end = datetime.now()
reflection_time = (reflection_end - reflection_start).total_seconds()

print(f"✅ Quality analysis completed in {reflection_time:.2f} seconds")
print("="*60)

# Display comprehensive reflection results
if console:
    # Rich formatted output for reflection analysis
    console.print("\n[bold magenta]🔍 REFLECTION & QUALITY ANALYSIS[/bold magenta]")
    
    # Quality metrics table
    quality_table = Table(show_header=True, header_style="bold cyan")
    quality_table.add_column("Quality Metric", style="yellow", width=25)
    quality_table.add_column("Score/Result", style="green")
    
    quality_table.add_row("Original Quality Score", f"{reflection_result.output.quality_score:.1%}")
    quality_table.add_row("Improvements Made", str(len(reflection_result.output.improvements_made)))
    quality_table.add_row("Missing Elements", str(len(reflection_result.output.missing_elements)))
    quality_table.add_row("Additional Insights", "✅ Provided" if reflection_result.output.additional_considerations else "❌ None")
    
    console.print(quality_table)
    
    # Enhanced answer panel
    console.print(Panel(
        reflection_result.output.reviewed_answer,
        title="[bold green]🎯 ENHANCED SALES ADVICE[/bold green]",
        border_style="green"
    ))
    
    # Improvements made
    if reflection_result.output.improvements_made:
        console.print("\n[bold blue]🔧 SPECIFIC IMPROVEMENTS MADE:[/bold blue]")
        for i, improvement in enumerate(reflection_result.output.improvements_made, 1):
            console.print(f"   {i}. {improvement}")
    
    # Missing elements identified
    if reflection_result.output.missing_elements:
        console.print("\n[bold red]⚠️  ELEMENTS THAT WERE MISSING:[/bold red]")
        for i, missing in enumerate(reflection_result.output.missing_elements, 1):
            console.print(f"   {i}. {missing}")
    
    # Additional considerations
    if reflection_result.output.additional_considerations:
        console.print(Panel(
            reflection_result.output.additional_considerations,
            title="[bold purple]💡 ADDITIONAL STRATEGIC CONSIDERATIONS[/bold purple]",
            border_style="purple"
        ))

else:
    # Fallback to standard print
    print("\n🔍 REFLECTION & QUALITY ANALYSIS:")
    print(f"Original Quality Score: {reflection_result.output.quality_score:.1%}")
    
    print(f"\n🎯 Enhanced Answer:")
    print(reflection_result.output.reviewed_answer)
    
    if reflection_result.output.improvements_made:
        print(f"\n🔧 Improvements Made:")
        for i, improvement in enumerate(reflection_result.output.improvements_made, 1):
            print(f"   {i}. {improvement}")
    
    if reflection_result.output.missing_elements:
        print(f"\n⚠️  Missing Elements:")
        for i, missing in enumerate(reflection_result.output.missing_elements, 1):
            print(f"   {i}. {missing}")
    
    if reflection_result.output.additional_considerations:
        print(f"\n💡 Additional Considerations:")
        print(reflection_result.output.additional_considerations)

print("\n" + "="*60)
print("✅ Dual-agent sales consultation completed!")

🔄 Running Reflection & Quality Analysis...
🔍 Analyzing primary response for quality and completeness...
✅ Quality analysis completed in 14.63 seconds


🔍 REFLECTION & QUALITY ANALYSIS

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Quality Metric            ┃ Score/Result ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Original Quality Score    │ 93.0%        │
│ Improvements Made         │ 3            │
│ Missing Elements          │ 2            │
│ Additional Insights       │ ✅ Provided  │
└───────────────────────────┴──────────────┘

╭─────────────────────────────────────────── 🎯 ENHANCED SALES ADVICE ────────────────────────────────────────────╮
│ Handling price objections effectively in enterprise B2B SaaS sales requires a strategic approach that focuses   │
│ on demonstrating value, understanding client needs, and negotiating terms that highlight the unique benefits of │
│ your solution. Here are several practical strategies you can use:                                               │
│                                                                                                                 │
│ 1. **Understand the Root Objection:** Begin by asking open-ended questions to uncover the underlying reasons    │
│ for their price concern. E.g., “Can you share more about what specifically seems too high compared to           │
│ competitors?” This helps identify whether the objection is based on budget, perceived value, or comparison with │
│ competitors.                                                                                                    │
│                                                                                                                 │
│ 2. **Value Proposition Emphasis:** Clearly articulate and align your solution’s value with the client’s         │
│ specific business outcomes. Showcase case studies or data that demonstrate the ROI and cost savings achieved by │
│ businesses similar to theirs. This helps shift focus from price to the overall business impact.                 │
│                                                                                                                 │
│ 3. **Competitive Differentiation:** Highlight unique features, support, integration capabilities, scalability,  │
│ and any bespoke solutions your product offers that competitors do not. This reinforces why choosing your        │
│ solution is economically advantageous in the long run.                                                          │
│                                                                                                                 │
│ 4. **Bundle and Incentivize:** Offer bundled packages or limited-time incentives that add value without         │
│ significant cost reduction. For example, including premium support for the first year can sweeten the deal and  │
│ justify the pricing.                                                                                            │
│                                                                                                                 │
│ 5. **Risk Reduction:** Propose terms that minimize the client’s perceived risk, such as trial periods,          │
│ performance guarantees, or flexible contracts that allow for growth and adjustments over time.                  │
│                                                                                                                 │
│ 6. **Negotiate based on volume or long-term commitment:** Offer price adjustments for larger volume purchases   │
│ or longer contractual commitments, ensuring the client sees it as a win-win scenario.                           │
│                                                                                                                 │
│ 7. **Build Strong Relationships:** Develop rapport and trust with clients through regular interactions,         │
│ allowing you to better understand their financial constraints and positively influence their purchasing         │
│ decisions over time.                                                                                            │
│                                                                                                                 │
│ 8. **Leverage Social Proof and Testimonials:** Provide testimonials and references from other enterprise        │
│ clients who testify to the value and cost-effectiveness of your solution. This can reassure potential clients   │
│ about the credibility and quality of your offering.    

🔧 SPECIFIC IMPROVEMENTS MADE:

...

⚠️  ELEMENTS THAT WERE MISSING:

1. Use of social proof and third-party endorsements.

2. Regular interaction examples for relationship building.

╭──────────────────────────────────── 💡 ADDITIONAL STRATEGIC CONSIDERATIONS ─────────────────────────────────────╮
│ Ensure all team members involved in the sales process are aligned on the value proposition and know how to      │
│ effectively communicate it in every client interaction. Training and role-playing scenarios can be beneficial.  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ Dual-agent sales consultation completed!


## 🚀 **Step 8: Try More Sales Scenarios**

Now that you've seen the dual-agent system in action, experiment with different sales scenarios! Replace the `sales_question` variable with any of these real-world challenges:

### 🎯 **Enterprise Sales Scenarios:**
- **Objection Handling**: "How do I handle the objection 'We need to think about it' when closing enterprise deals?"
- **Stakeholder Management**: "What's the best approach for navigating multiple decision-makers in complex B2B sales cycles?"
- **Competitive Positioning**: "How do I effectively compete against incumbents when our solution is newer to market?"

### 📞 **Prospecting & Lead Generation:**
- **Cold Outreach**: "What are the most effective cold email strategies for SaaS sales in 2024?"
- **LinkedIn Prospecting**: "How can I optimize my LinkedIn outreach for high-value B2B prospects?"
- **Referral Programs**: "What's the best way to build and scale a customer referral program?"

### 🤝 **Relationship Building:**
- **Trust Building**: "How do I quickly establish credibility and trust with C-level executives?"
- **Long-term Relationships**: "What strategies work best for maintaining relationships with existing clients to drive expansion revenue?"
- **Customer Success**: "How can I align sales and customer success to reduce churn and increase upsells?"

### 📊 **Sales Strategy & Process:**
- **Sales Methodology**: "Should I use SPIN, Challenger, or MEDDIC methodology for enterprise software sales?"
- **Pipeline Management**: "How do I effectively qualify leads and manage a complex sales pipeline?"
- **Performance Metrics**: "What are the most important sales KPIs I should track for SaaS sales success?"

### 💰 **Negotiation & Closing:**
- **Pricing Strategy**: "How do I present value-based pricing to justify premium costs?"
- **Contract Negotiations**: "What are effective strategies for negotiating terms with procurement teams?"
- **Closing Techniques**: "What are modern, relationship-focused closing techniques that actually work?"

---

## 🔧 **Customization Tips:**

### **Modify Agent Behavior:**
```python
# Adjust model temperature for more creative responses
model_temperature = 0.9  # Higher = more creative

# Change to GPT-4 for enhanced reasoning
model_name = "gpt-4"

# Customize system prompts for specific industries
sales_agent.system_prompt += "Focus specifically on healthcare technology sales."
```

### **Add Your Own Schemas:**
```python
class IndustrySpecificAnswer(BaseModel):
    answer: str
    industry_considerations: List[str]
    compliance_notes: Optional[str]
    # Add fields specific to your industry
```

### **Track Agent Performance:**
```python
# Log all interactions for analysis
interaction_log = {
    "timestamp": datetime.now(),
    "question": sales_question,
    "primary_confidence": primary_result.output.confidence_score,
    "reflection_quality": reflection_result.output.quality_score,
    "improvements_count": len(reflection_result.output.improvements_made)
}
```

---

## 🎉 **Next Steps:**

1. **Experiment** with different sales scenarios
2. **Customize** the agents for your specific industry or role
3. **Track** agent performance and improve prompts
4. **Integrate** into your sales workflow or training programs
5. **Scale** by adding more specialized agents for different sales functions
